In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mean_squared_error
from imblearn.over_sampling import SMOTE
#여기서 주의할 점은
#SMOTE를 적용할 때는 반드시 학습 데이터 세트만 오버샘플링을 해야합니다!!!
# 검증 데이터 세트 혹은 테스트 데이터 세트를 오버샘플링 하는 경우 결국 원본 데이터가 아닌 데이터 세트에서 검증되기 때문에 올바른 검증이 되지 않습니다. 
#https://coding-potato.tistory.com/17

from rdkit import DataStructs
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors

from typing import List, Union

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
def morgan_binary_features_generator(mol: Union[str, Chem.Mol], plot_img = False,
                                     radius: int = 6,
                                     num_bits: int = 4096) -> np.ndarray:
    
    mol = Chem.MolFromSmiles(mol) if type(mol) == str else mol
    if plot_img:
        display(mol)
    
    features_vec = AllChem.GetHashedMorganFingerprint(mol, radius, nBits=num_bits)
    features = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(features_vec, features)

    return features

In [ ]:
def getMolDescriptors(mol: Union[str, Chem.Mol], missingVal=None):
    ''' calculate the full list of descriptors for a molecule

        missingVal is used if the descriptor cannot be calculated
    '''
    mol = Chem.MolFromSmiles(mol) if type(mol) == str else mol
    res = {}
    for nm,fn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=-1):
        self.patience = patience  # number of times to allow for no improvement before stopping the execution
        self.min_delta = min_delta  # the minimum change to be counted as improvement: if it is negative means, lower value is good
        self.counter = 0  # count the number of times the validation accuracy not improving
        self.min_validation_loss = np.inf

    # return True when encountering _patience_ times decrease in validation loss 
    def __call__(self, validation_loss, verbose=False):
        if ((validation_loss+self.min_delta) < self.min_validation_loss):
            self.min_validation_loss = validation_loss
            self.counter = 0  # reset the counter if validation loss decreased at least by min_delta
            
        elif ((validation_loss+self.min_delta) > self.min_validation_loss):
            self.counter += 1 # increase the counter if validation loss is not decreased by the min_delta
            if verbose:
                print(f"  >> now{validation_loss:.3f} > best{self.min_validation_loss:.3f}")
            if self.counter >= self.patience:
                return True
        return False

In [5]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

train_df.drop_duplicates(["SMILES"], inplace=True)
train_df = train_df.reset_index(drop=True)

train_descriptor = pd.DataFrame([getMolDescriptors(smile) for smile in train_df['SMILES']])
test_descriptor =  pd.DataFrame([getMolDescriptors(smile) for smile in test_df['SMILES']])

train_df = pd.concat([train_df, train_descriptor], axis=1)
test_df = pd.concat([test_df, test_descriptor], axis=1)

train_df.drop(columns=['MolWt', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'MolLogP'], axis=1, inplace=True)
test_df.drop(columns=['MolWt', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'MolLogP'], axis=1, inplace=True)

train_df.fillna(train_df.mean(numeric_only=True), inplace=True)
test_df.fillna(test_df.mean(numeric_only=True), inplace=True)

train_df=train_df.drop(train_df.loc[train_df["MLM"]>100].index).reset_index(drop=True)
train_df=train_df.drop(train_df.loc[train_df["HLM"]>100].index).reset_index(drop=True)


train_df["AlogP"] = np.where(pd.isna(train_df["AlogP"]), train_df["LogD"], train_df["AlogP"])
test_df["AlogP"] = np.where(pd.isna(test_df["AlogP"]), test_df["LogD"], test_df["AlogP"])

smote = SMOTE(sampling_strategy='auto')
train_MLM_x, train_MLM_y = smote.fit_resample(train_df.drop(columns=['id','SMILES', "HLM"]), train_df["MLM"].apply(lambda x: np.int8(x//10)))
train_HLM_x, train_HLM_y = smote.fit_resample(train_df.drop(columns=['id','SMILES', "MLM"]), train_df["HLM"].apply(lambda x: np.int8(x//10)))

train_MLM = TabularDataset(train_MLM_x)
train_HLM = TabularDataset(train_HLM_x)
test = TabularDataset(test_df.drop(["id"], axis=1))

In [ ]:
train_MLM_y.value_counts()

In [ ]:
from IPython.display import Image, display

In [ ]:
predictor_MLM = TabularPredictor(label='MLM', eval_metric='root_mean_squared_error', verbosity=False).fit(train_MLM)
predictor_HLM = TabularPredictor(label='HLM', eval_metric='root_mean_squared_error', verbosity=False).fit(train_HLM)

ld_board_MLM = predictor_MLM.leaderboard(train_MLM, silent=True)
print("="*20, "MLM", "="*20)
print(ld_board_MLM)
print(f"Best: {predictor_MLM.get_model_best()}")

ld_board_HLM = predictor_HLM.leaderboard(train_HLM, silent=True)
print("="*20, "HLM", "="*20)
print(ld_board_HLM)
print(f"Best: {predictor_HLM.get_model_best()}")

In [ ]:
path_to_png=predictor_MLM.plot_ensemble_model()
display(Image(filename=path_to_png))
path_to_png=predictor_HLM.plot_ensemble_model()
display(Image(filename=path_to_png))

In [ ]:
# 결과파일 작성
pred_MLM = predictor_MLM.predict(test)
pred_HLM = predictor_HLM.predict(test)

submission = pd.DataFrame()
submission["id"] = test_df["id"]
submission["MLM"] = pred_MLM
submission["HLM"] = pred_HLM

submission.to_csv("../output/submission.csv", index=False)

In [ ]:
# check before sub
submission.isna().sum()

In [ ]:
submission